In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [10]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [11]:
def string_to_onehot(string):
    start=np.zeros(shape=len(char_list),dtype=int)
    end=np.zeros(shape=len(char_list),dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_letters,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero])
    output=np.vstack([start,end])
    return output

In [12]:
# 원-핫 벡터를 다시 문자로 바꾸는 부분
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()

    def forward(self, input, hidden):
        #입력과 hidden state를 cat으로 붙인다.
        combined = torch.cat((input, hidden), 1)
        #붙인 값을 i2h 및 i2o에 통과시켜 hidden state는 업데이트, 결과값은 계산.
        hidden = self.act_fn(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1,self.hidden_size)
    
rnn = RNN(n_letters,n_hidden,n_letters)

In [14]:
#손실함수와 최적화함수 설정

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(),lr=lr)

In [15]:
#train

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    optimizer.zero_grad()
    #학습전 hidden state 초기화
    hidden = rnn.init_hidden()
    
    total_loss = 0
    for j in range(one_hot.size()[0]-1):
        #입력은 앞에 글자
        #pytorch에서 p y t o r c
        input_ = one_hot[j:j+1,:]
        #목표값은 뒤에 글자
        #pytorch에서 y t o r c h
        target = one_hot[j+1]
        output, hidden = rnn.forward(input_,hidden)
        
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
    
    total_loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(total_loss)

tensor(1.8541, grad_fn=<AddBackward0>)
tensor(0.7175, grad_fn=<AddBackward0>)
tensor(0.3747, grad_fn=<AddBackward0>)
tensor(0.2023, grad_fn=<AddBackward0>)
tensor(0.1108, grad_fn=<AddBackward0>)
tensor(0.0710, grad_fn=<AddBackward0>)
tensor(0.0427, grad_fn=<AddBackward0>)
tensor(0.0275, grad_fn=<AddBackward0>)
tensor(0.0179, grad_fn=<AddBackward0>)
tensor(0.0130, grad_fn=<AddBackward0>)
tensor(0.0080, grad_fn=<AddBackward0>)
tensor(0.0054, grad_fn=<AddBackward0>)
tensor(0.0038, grad_fn=<AddBackward0>)
tensor(0.0035, grad_fn=<AddBackward0>)
tensor(0.0018, grad_fn=<AddBackward0>)
tensor(0.0009, grad_fn=<AddBackward0>)
tensor(0.0006, grad_fn=<AddBackward0>)
tensor(0.0004, grad_fn=<AddBackward0>)
tensor(0.0002, grad_fn=<AddBackward0>)
tensor(0.0002, grad_fn=<AddBackward0>)
tensor(0.0002, grad_fn=<AddBackward0>)
tensor(0.0002, grad_fn=<AddBackward0>)
tensor(7.8438e-05, grad_fn=<AddBackward0>)
tensor(6.1470e-05, grad_fn=<AddBackward0>)
tensor(0.0002, grad_fn=<AddBackward0>)
tensor(0.0004, gr

In [18]:
# test 
# hidden state 는 처음 한번만 초기화해줍니다.

start = torch.zeros(1,n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    # 처음 입력으로 start token을 전달해줍니다.
    input_ = start
    # output string에 문자들을 계속 붙여줍니다.
    output_string = ""

    # 원래는 end token이 나올때 까지 반복하는게 맞으나 끝나지 않아서 string의 길이로 정했습니다.
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        # 결과값을 문자로 바꿔서 output_string에 붙여줍니다.
        output_string += onehot_to_word(output.data)
        # 또한 이번의 결과값이 다음의 입력값이 됩니다.
        input_ = output

print(output_string)

hello pytorch. how long can a rnn cell remember?
